## Analyse descriptive avec GraphLab Create

In [143]:
import graphlab as gl
import re
from datetime import datetime

time: 1.28 ms


#### Pour mésurer le temps d'opération pour chaque cellule du notebook:

In [144]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 925 ms


Lecture de la base des crimes à San Francisco : 

In [258]:
sfpd = gl.SFrame.read_csv("SFPD.csv",column_type_hints=[int,str,str,str,str,str,str,str,str,str,str,str,int])

Read 255190 lines. Lines per second: 76307.9

Read 1277673 lines. Lines per second: 142907

Finished parsing file /home/user3/SFPD.csv

Parsing completed. Parsed 1957852 lines in 11.9846 secs.

time: 12.6 s


On commence par générer automatiquement la description pour chaque colonne en fonction de son type.

In [8]:
sfpd.show()

Canvas is accessible via web browser at the URL: http://localhost:40032/index.html
Opening Canvas in default web browser.


ERROR:tornado.access:500 GET /sketch/%EF%BB%BFIncidntNum (127.0.0.1) 34.86ms


Pour afficher les viz de Graphlab directement dans le Notebook : 

In [146]:
gl.canvas.set_target('ipynb')

time: 801 µs


Pour retourner à l'affichage complet : 

In [283]:
# gl.canvas.set_target('browser')

time: 2.15 ms


# Description de la base de données : 

In [151]:
sfpd.num_rows

<bound method SFrame.num_rows of Columns:
	﻿IncidntNum	int
	Category	str
	Descript	str
	DayOfWeek	str
	Date	str
	Time	str
	PdDistrict	str
	Resolution	str
	Address	str
	X	float
	Y	float
	Location	str
	PdId	int

Rows: 1957852

Data:
+-------------+----------------+------------------------------+-----------+
| ﻿IncidntNum |    Category    |           Descript           | DayOfWeek |
+-------------+----------------+------------------------------+-----------+
|  100475254  |    ASSAULT     |           BATTERY            |   Friday  |
|  100475254  |    WARRANTS    |        WARRANT ARREST        |   Friday  |
|  100475254  |  NON-CRIMINAL  | AIDED CASE, MENTAL DISTURBED |   Friday  |
|  110914565  |    WARRANTS    |        WARRANT ARREST        |   Friday  |
|   11351210  | MISSING PERSON |         FOUND PERSON         |  Tuesday  |
|  120789988  | MISSING PERSON |         FOUND PERSON         |   Monday  |
|  140608681  |    ROBBERY     |    ROBBERY, BODILY FORCE     |  Thursday |
|  140608

time: 87.4 ms


In [149]:
sfpd['Date'].show()

time: 1.15 s


In [152]:
sfpd.column_names()

['\xef\xbb\xbfIncidntNum',
 'Category',
 'Descript',
 'DayOfWeek',
 'Date',
 'Time',
 'PdDistrict',
 'Resolution',
 'Address',
 'X',
 'Y',
 'Location',
 'PdId']

time: 4.3 ms


### Données de l'année 2016:

par adresse

In [76]:
sfpd['Address'][sfpd['Date'].contains('2016')].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      | 95343 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  | 13952 |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------------------+------------------------+--------------------------+
| value | 800 Block of BRYANT ST | 800 Block of MARKET ST | 1000 Block of POTRERO AV |
+-------+------------------------+------------------------+--------------------------+
| count |          2230          |          857           |           408            |
+-------+------------------------+------------------------+--------------------------+
+------------------------+-----------------------------+------------------------------+
| 900 Block of MARKET ST | 0 Block of UNITEDNATIONS PZ | 500 Block of JOHNFKENNEDY DR |
+------------------------+-----------------------------+------------------------------

In [153]:
sfpd['Address'].sketch_summary()


+------------------+---------+----------+
|       item       |  value  | is exact |
+------------------+---------+----------+
|      Length      | 1957852 |   Yes    |
| # Missing Values |    0    |   Yes    |
| # unique values  |  24914  |    No    |
+------------------+---------+----------+

Most frequent items:
+-------+------------------------+------------------------+--------------------------+
| value | 800 Block of BRYANT ST | 800 Block of MARKET ST | 2000 Block of MISSION ST |
+-------+------------------------+------------------------+--------------------------+
| count |         58248          |         15238          |          10490           |
+-------+------------------------+------------------------+--------------------------+
+--------------------------+------------------------+--------------------+-------------------+
| 1000 Block of POTRERO AV | 900 Block of MARKET ST | 0 Block of TURK ST | 0 Block of 6TH ST |
+--------------------------+------------------------+-----

time: 1.39 s


In [75]:
sfpd['PdDistrict'].show()

## Split le champs "Date" en année - mois - jour : 

In [154]:
sfpd['Year']=sfpd['Date'].apply(lambda x : datetime.strptime(x, '%m/%d/%Y').year)
sfpd['Month']=sfpd['Date'].apply(lambda x : datetime.strptime(x, '%m/%d/%Y').month)
sfpd['Day']=sfpd['Date'].apply(lambda x : datetime.strptime(x, '%m/%d/%Y').day)

time: 6.46 s


In [155]:
sfpd['Year','Month','Day'].show()

time: 46.4 s


In [118]:
sfpd[sfpd['Address'].contains('800 Block of BRYANT ST')].show()

## Extraction de la liste des coordonnées uniques : 

In [54]:
unique_coords = list(set(sfpd['Location']))

time: 2.65 s


In [57]:
unique_coords

['(37.7638126844306, -122.456670565112)',
 '(37.7806243476395, -122.509879755443)',
 '(37.7518870787623, -122.508238363207)',
 '(37.7422553063427, -122.402289971911)',
 '(37.754404748573, -122.405248327082)',
 '(37.7150736699192, -122.448458098344)',
 '(37.7375161241247, -122.418860969819)',
 '(37.7720402284409, -122.414148604934)',
 '(37.720467659541, -122.397490733097)',
 '(37.7659226957027, -122.427661930817)',
 '(37.7217937925358, -122.397014793661)',
 '(37.721435763041, -122.436724043712)',
 '(37.7651192951994, -122.407544906452)',
 '(37.7216178242752, -122.435066260529)',
 '(37.7284180706602, -122.450000790445)',
 '(37.7957458265093, -122.407107913789)',
 '(37.7727842765697, -122.478297956235)',
 '(37.8014549885662, -122.404638077211)',
 '(37.7813151199526, -122.455389197974)',
 '(37.7745652547201, -122.501598092064)',
 '(37.7788663496622, -122.489953332298)',
 '(37.7542717530655, -122.437129000157)',
 '(37.731702030339, -122.483658105938)',
 '(37.7341714275463, -122.387911252085

time: 39.9 ms


In [139]:
len(unique_coords)

37097

time: 3.64 ms


## Scraping des Zip codes avec l'API GeoNames

In [141]:
import re
import xml.etree.ElementTree as ET
import urllib2
from xml.dom import minidom
import time
import csv

# time.sleep(5)  - 5 sec pause d'execution du code

time: 2.11 ms


API GeoNames est limité à 2000 requètes par heure par utilisateur. Il faut modifier le code ci-dessous en conséquence si besoin de réconstituer toute la base.

In [61]:
#response = urllib2.urlopen(apilink)
#apixml = response.read()

In [3]:
#apixml

In [68]:
#xmldoc = minidom.parseString(apixml)

In [1]:
#xmldoc

In [72]:
#obs_values = xmldoc.getElementsByTagName('postalcode')

In [2]:
#print obs_values[0].firstChild.nodeValue

In [44]:
#postalcodes = [minidom.parseString(urllib2.urlopen("http://api.geonames.org/findNearbyPostalCodes?lat="+lat+"&lng="+lng+"&maxRows=1&username=linnlinn").read()).getElementsByTagName('postalcode')[0].firstChild.nodeValue for lat, lng in zip(X,Y)]

In [48]:
#postCodesList = zip(X, Y, postalcodes)

time: 2.81 ms


In [123]:
#username = 'fitec'
username = 'linnlinn'
print('userName=%s' % username)

userName=linnlinn
time: 1.08 ms


In [129]:
#time.sleep(3000)
#utiliser sleep pour éviter de dépasser la limite de l'API
for i in range(0,37):
    test_list = unique_coords[i*1000:(i+1)*1000]
    X = [re.search("^\((\d+\.*\d*),",element).group(1) for element in test_list]
    Y = [re.search("(-\d+\.*\d*)\)",element).group(1) for element in test_list]
    postalcodes = [minidom.parseString(urllib2.urlopen("http://api.geonames.org/findNearbyPostalCodes?lat="+lat+"&lng="+lng+"&maxRows=1&username=%s" % username).read()).getElementsByTagName('postalcode')[0].firstChild.nodeValue for lat, lng in zip(X,Y)]
    postCodesList = zip(X, Y, postalcodes)
    with open('postcodes%i.csv' % i, 'wb') as csvfile:
        wr = csv.writer(csvfile, delimiter=',')
        fieldnames = ['X', 'Y', 'ZipCode']
        wr.writerow(fieldnames)
        [wr.writerow(row) for row in postCodesList]
        csvfile.close()

time: 44.2 s


## Croisement des données

Lecture des fichiers zipcodes un après l'autre et merge:

In [261]:
zipcodesDF = gl.SFrame.read_csv('postcodes0.csv',column_type_hints=[str,str,str])
for i in range(1,38):
    tempDF = gl.SFrame.read_csv('postcodes%i.csv' % i,column_type_hints=[str,str,str])
    zipcodesDF = zipcodesDF.append(tempDF)


Finished parsing file /home/user3/postcodes0.csv

Parsing completed. Parsed 1000 lines in 0.113344 secs.

Finished parsing file /home/user3/postcodes1.csv

Parsing completed. Parsed 1000 lines in 0.052836 secs.

Finished parsing file /home/user3/postcodes2.csv

...

Finished parsing file /home/user3/postcodes36.csv

Parsing completed. Parsed 1000 lines in 0.028644 secs.

Finished parsing file /home/user3/postcodes37.csv

Parsing completed. Parsed 97 lines in 0.031261 secs.

time: 1.81 s


In [234]:
zipcodesDF.show()

time: 147 ms


In [262]:
locDF=zipcodesDF.pack_columns(columns=['X','Y'],new_column_name="Location")
zipcodesDF['Location']=locDF['Location']
zipcodesDF['Location']=zipcodesDF['Location'].apply(lambda x: "("+ x[0]+", "+x[1]+")")
locDF['Location']=locDF['Location'].apply(lambda x: "("+ x[0]+", "+x[1]+")")

time: 56.3 ms


time: 16.2 ms


In [263]:
locDF.show

<bound method SFrame.show of Columns:
	ZipCode	str
	Location	str

Rows: 37097

Data:
+---------+-------------------------------+
| ZipCode |            Location           |
+---------+-------------------------------+
|  94131  | (37.7638126844306, -122.45... |
|  94121  | (37.7806243476395, -122.50... |
|  94122  | (37.7518870787623, -122.50... |
|  94124  | (37.7422553063427, -122.40... |
|  94110  | (37.754404748573, -122.405... |
|  94112  | (37.7150736699192, -122.44... |
|  94110  | (37.7375161241247, -122.41... |
|  94103  | (37.7720402284409, -122.41... |
|  94124  | (37.720467659541, -122.397... |
|  94114  | (37.7659226957027, -122.42... |
+---------+-------------------------------+
[37097 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.>

time: 32.2 ms


In [275]:
sfpd_zip=sfpd.join(locDF, on=['Location'], how='outer')

time: 10.5 s


In [276]:
sfpd_zip.show

<bound method SFrame.show of Columns:
	﻿IncidntNum	int
	Category	str
	Descript	str
	DayOfWeek	str
	Date	str
	Time	str
	PdDistrict	str
	Resolution	str
	Address	str
	X	str
	Y	str
	Location	str
	PdId	int
	ZipCode	str

Rows: 1957852

Data:
+-------------+----------------+------------------------------+-----------+
| ﻿IncidntNum |    Category    |           Descript           | DayOfWeek |
+-------------+----------------+------------------------------+-----------+
|  100475254  |    ASSAULT     |           BATTERY            |   Friday  |
|  100475254  |    WARRANTS    |        WARRANT ARREST        |   Friday  |
|  100475254  |  NON-CRIMINAL  | AIDED CASE, MENTAL DISTURBED |   Friday  |
|  110914565  |    WARRANTS    |        WARRANT ARREST        |   Friday  |
|   11351210  | MISSING PERSON |         FOUND PERSON         |  Tuesday  |
|  120789988  | MISSING PERSON |         FOUND PERSON         |   Monday  |
|  140608681  |    ROBBERY     |    ROBBERY, BODILY FORCE     |  Thursday |
|  1

time: 149 ms


In [277]:
sfpd_zip['ZipCode'].sketch_summary()


+------------------+---------+----------+
|       item       |  value  | is exact |
+------------------+---------+----------+
|      Length      | 1957852 |   Yes    |
| # Missing Values |    0    |   Yes    |
| # unique values  |    30   |    No    |
+------------------+---------+----------+

Most frequent items:
+-------+--------+--------+--------+--------+--------+--------+-------+-------+
| value | 94103  | 94102  | 94110  | 94109  | 94124  | 94112  | 94117 | 94107 |
+-------+--------+--------+--------+--------+--------+--------+-------+-------+
| count | 285684 | 241434 | 204472 | 151977 | 131835 | 101589 | 82231 | 79772 |
+-------+--------+--------+--------+--------+--------+--------+-------+-------+
+-------+-------+
| 94115 | 94133 |
+-------+-------+
| 77925 | 67852 |
+-------+-------+


time: 524 ms


In [316]:
set(sfpd_zip['ZipCode'])

{'94014',
 '94015',
 '94102',
 '94103',
 '94104',
 '94105',
 '94107',
 '94108',
 '94109',
 '94110',
 '94111',
 '94112',
 '94114',
 '94115',
 '94116',
 '94117',
 '94118',
 '94121',
 '94122',
 '94123',
 '94124',
 '94127',
 '94129',
 '94130',
 '94131',
 '94132',
 '94133',
 '94134',
 '94158',
 'H0H'}

time: 1.02 s


In [320]:
sfpd_zip.export_csv('~/sfpd_zip.csv')

time: 17.2 s


### Jointure avec les données du scraping : 

In [306]:
scrapDF = gl.SFrame.read_csv('~/SF_scrap_data.csv',column_type_hints=[int,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,str])

Read 26 lines. Lines per second: 3198.03

Finished parsing file /home/user3/SF_scrap_data.csv

Parsing completed. Parsed 26 lines in 0.008217 secs.

time: 19.9 ms


In [307]:
scrapDF.show

<bound method SFrame.show of Columns:
	X1	int
	American Indian population	float
	Asian population	float
	Average Adjusted Gross Income (AGI) in 2012	float
	Average household size	float
	Black population	float
	Estimated zip code population in 2013	float
	Females	float
	Hispanic or Latino population	float
	Houses and condos	float
	Land area	float
	Males	float
	Mar. 2016 cost of living index in zip code	float
	Median resident age	float
	Native Hawaiian and Other Pacific Islander population	float
	Population density (people per square mile)	float
	Renter-occupied apartments	float
	Residents with income below 50% of the poverty level in 2013	float
	Salary/wage	float
	Some other race population	float
	Two or more races population	float
	Water area	float
	White population	float
	Zip code population in 2000	float
	Zip code population in 2010	float
	ZipCode	str

Rows: 26

Data:
+----+----------------------------+------------------+-------------------------------+
| X1 | American Indian populat

time: 146 ms


In [311]:
scrapDF['ZipCode'].show()

Canvas is updated and available in a tab in the default browser.
time: 7.1 ms


In [312]:
sfpd_total=sfpd_zip.join(scrapDF, on=['ZipCode'], how='outer')

time: 14.5 s


In [313]:
sfpd_total.show()

Canvas is accessible via web browser at the URL: http://localhost:43475/index.html
Opening Canvas in default web browser.
time: 426 ms


ERROR:tornado.access:500 GET /sketch/%EF%BB%BFIncidntNum (127.0.0.1) 443.03ms
ERROR:tornado.access:500 GET /sketch/%EF%BB%BFIncidntNum (127.0.0.1) 42.32ms


In [321]:
sfpd_total.export_csv('~/sfpd_total.csv')

time: 1min 58s
